In [202]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [203]:
# Load the data from a CSV file
df = pd.read_csv("../data/all_v2_data.csv", sep=",")
df_pred= pd.read_csv("../data/september_to_predict.csv", sep=",")

In [204]:
df.dropna(inplace=True)

In [205]:
df_pred

,Mnd,Ap,Hoyre,Frp,SV,SP,KrF,Venstre,MDG,Rodt,...,Rodt_reg_lag3,Rodt_reg_lag6,Andre_reg_lag3,Andre_reg_lag6,ledighet,ledig_trend3,ledig_trend6,styringsrente,styringsrente_trend3,styringsrente_trend6
0,9/30/2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,4,0,0,4,0,-0.033333


In [206]:
parties=['Ap', 'Hoyre', 'Frp', 'SV', 'SP', 'KrF', 'Venstre', 'MDG','Rodt', 'Andre']
predictions=[]

In [207]:
for party in parties:
    y=df[[party]]
    X=df[[f'{party}_var', f'{party}_skandale',f'{party}_skandale_lag3', f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    x_pred=df_pred[[f'{party}_var', f'{party}_skandale',f'{party}_skandale_lag3', f'{party}_skandale_lag6', 
          f'{party}_reg', f'{party}_reg_lag3',f'{party}_reg_lag6', 
       'ledighet', 'ledig_trend3', 'ledig_trend6', 'styringsrente',
       'styringsrente_trend3', 'styringsrente_trend6']]
    
    x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42
    )
    pipeline = Pipeline([
    ("feature_selection", SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), threshold="median")),
    ("regressor", RandomForestRegressor(n_estimators=10000, random_state=42))
    ])
    pipeline.fit(x_train, y_train.values.ravel())
    y_pred = pipeline.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    
    
    p=pipeline.predict(x_pred)
    print(party + ": " + str(p[0]))
    predictions.append({'party': party, 'prediction': p[0], "r2_score": r2})

Ap: 30.495130000002128
Hoyre: 17.329289999999876
Frp: 19.31503999999855
SV: 6.5467300000007045
SP: 7.048320000000186
KrF: 3.392479999999848
Venstre: 3.9623699999999205
MDG: 3.431789999999881
Rodt: 4.856029999999859
Andre: 4.597609999999709


In [208]:
pred_df = pd.DataFrame(predictions)
total = pred_df['prediction'].sum()
pred_df['prediction'] = (pred_df['prediction'] / total) * 100

# Optional: Round to 2 decimals for clarity
pred_df['prediction'] = pred_df['prediction'].round(2)

In [209]:
pred_df

,party,prediction,r2_score
0,Ap,30.20,0.894634
1,Hoyre,17.16,0.933938
2,Frp,19.13,0.930505
3,SV,6.48,0.972560
4,SP,6.98,0.988457
5,KrF,3.36,0.946200
6,Venstre,3.92,0.687998
7,MDG,3.40,0.953052
8,Rodt,4.81,0.985373
9,Andre,4.55,0.985390
